测试使用： https://github.com/truera/trulens/issues/1215

根据bug回复写出一个原型，但是依然是 `qs_relevance None`

In [1]:
%%time

import os

base_url = "http://ape:3000/v1"
api_key = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = base_url

CPU times: user 10 µs, sys: 2 µs, total: 12 µs
Wall time: 12.9 µs


In [2]:
import pandas as pd

data = {
    'prompt': ['Where is Germany?', 'What is the capital of France?'],
    'response': ['Germany is in Europe', 'The capital of France is Paris'],
    'context': ['Germany is a country located in Europe.', 'France is a country in Europe and its capital is Paris.']
}
df = pd.DataFrame(data)
df.head()


,prompt,response,context
0,Where is Germany?,Germany is in Europe,Germany is a country located in Europe.
1,What is the capital of France?,The capital of France is Paris,France is a country in Europe and its capital ...


In [3]:
%%time

from trulens_eval.schema.feedback import Select
from trulens_eval.tru_virtual import VirtualApp
from trulens_eval.tru_virtual import VirtualRecord

virtual_app = dict(
    llm=dict(
        modelname="some llm component model name"
    ),
    template="information about the template I used in my app",
    debug="all of these fields are completely optional"
)

virtual_app = VirtualApp(virtual_app) # can start with the prior dictionary
virtual_app[Select.RecordCalls.llm.maxtokens] = 1024

retriever_component = Select.RecordCalls.retriever
virtual_app[retriever_component] = "this is the retriever component"

context_call = retriever_component.get_context

data_dict = df.to_dict('records')

data = []

for record in data_dict:
    rec = VirtualRecord(
        main_input=record['prompt'],
        main_output=record['response'],
        calls=
            {
                context_call: dict(
                    args=[record['prompt']],
                    rets=[record['context']]
                )
            }
        )
    data.append(rec)

CPU times: user 5.83 s, sys: 493 ms, total: 6.32 s
Wall time: 6.14 s


In [9]:
%%time

from trulens_eval.feedback.feedback import Feedback
from trulens_eval.feedback.provider import OpenAI
# from dotenv import load_dotenv
# load_dotenv()
import os

fopenai = OpenAI(
    base_url = "http://ape:3000",
    api_key = api_key
)

# Select context to be used in feedback. We select the return values of the
# virtual `get_context` call in the virtual `retriever` component. Names are
# arbitrary except for `rets`.
context = context_call.rets[:]

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance)
    .on_input()
    .on(context)
)

✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input context will be set to __record__.app.retriever.get_context.rets[:] .
CPU times: user 54.6 ms, sys: 0 ns, total: 54.6 ms
Wall time: 53.5 ms


In [10]:
%%time

from trulens_eval.tru_virtual import TruVirtual

virtual_recorder = TruVirtual(
    app_id="a virtual app",
    app=virtual_app,
    feedbacks=[f_context_relevance]
)

temp = []

for record in data:
    result = virtual_recorder.add_record(record=record)
    temp.append(result)

for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)
    print('hello')

qs_relevance None
hello
CPU times: user 355 ms, sys: 4.55 ms, total: 359 ms
Wall time: 14.3 s
